#### this
Try cache train data one more time

In [1]:
import pandas as pd
import xgboost as xgb
from xgboost import XGBClassifier
import datetime; import pytz
import matplotlib.pyplot as plt
from scipy.special import softmax
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split # (*arrays, **options)
import numpy as np
from sklearn.metrics import log_loss
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

from joblib import dump, load
import joblib
import os
from sklearn.metrics import confusion_matrix, mean_squared_error
from sklearn.model_selection import KFold, train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score
import fresh.utils as fu

from importlib import reload
from collections import Counter
from tqdm.notebook import tqdm
import fresh.preproc.v1 as pv1

In [2]:
datadir = '/opt/data'
localdir = '/opt/program'


tripsdf = pd.read_csv(f'{datadir}/2013-07 - Citi Bike trip data.csv'
                     )#.sample(frac=0.017, random_state=42)
stationsdf = pd.read_csv(f'{localdir}/datas/stations/stations-2018-12-04-c.csv',
                        index_col=0)


In [3]:
X, y, neighborhoods = fu.prepare_data(tripsdf, stationsdf)

# ... actually doing this part jsut to get those labels... 
X_train, X_test, y_train, y_test = train_test_split(X, y)
# preproc
(X_transformed,
     one_hot_enc, le,
     y_enc) = pv1.preprocess(X_train, y_train, # X[train_index]
                         neighborhoods)
labels = le.classes_


In [4]:
print(f'mem, ({fu.get_my_memory()})')

mem, ({'pmem': '32.7', 'rss': '0.64 GiB'})


In [5]:
def prepare_dtrain(X, y, workdir, i):
    
    outpath = f'{workdir}/dtrain.txt'
    fu.save_libsvm(X.toarray(), y, outpath)
    fu.log(workdir, f'[{i}] Saved train data: {outpath}')
    dtrain = xgb.DMatrix(f'{outpath}#dtrain.cache')
    return dtrain

def prepare_dtest(X, workdir, i):
    outpath = f'{workdir}/dtest.txt'    
    fu.save_libsvm(X, outpath=outpath)
    fu.log(workdir, f'[{i}] Done saving to {outpath}')
    dtest = xgb.DMatrix(f'{outpath}') # #dtest.cache
    return dtest

In [6]:
%%time

workdir = fu.make_work_dir(); print('workdir: ', workdir)
fu.log(workdir, 'Starting', f'mem, ({fu.get_my_memory()})')

params = {'max_depth':3, 
          'learning_rate': .1, # 'eta':0.1   # alias
          'objective':'multi:softprob',   # mlogloss? 
          'num_class': len(labels), # 54 
          'base_score':0.5, 
          'booster':'gbtree', 
          'colsample_bylevel':1,
          'colsample_bynode':1, 
          'colsample_bytree':1, 
          'gamma':0,
          'max_delta_step':0, 
          'min_child_weight':1, #'missing':nan, 
          'random_state':0,
          'reg_alpha':0, 
          'reg_lambda':1,
          'scale_pos_weight':1, 
          'seed': 42,
          #'silent':None, 
          'subsample':1, 
          'verbosity': 2
          
          # from sklearn...
          # 'n_estimators':100, 'n_jobs':1,
         }
rng = np.random.RandomState(31337)

kf = KFold(n_splits=2, shuffle=True, random_state=rng)
for (i, (train_index, test_index)) in enumerate(kf.split(X)):    


    # preprocess
    (X_transformed, one_hot_enc, le,
         y_enc) = pv1.preprocess(X[train_index], y[train_index], 
                             neighborhoods)
    fu.log(workdir, f'[{i}] Done preprocessing', 
                   f'mem, ({fu.get_my_memory()})')
    labels = le.classes_

    dtrain = prepare_dtrain(X_transformed, y_enc, workdir, i)

    watchlist = [(dtrain, 'train')]
    num_round = 100

    xgb_model = xgb.train(params, dtrain, num_round, watchlist)
    fu.log(workdir, f'[{i}] Done xgb.train', 
                   f'mem, ({fu.get_my_memory()})')

    # test..
    X_test_transformed = one_hot_enc.transform(
        X[test_index]).toarray()
    actuals = le.transform(y[test_index])
    fu.log(workdir, f'[{i}] Done transforming test data')
    
    dtest = prepare_dtest(X_test_transformed, workdir, i)
    y_prob_vec = xgb_model.predict(dtest)
    predictions = np.argmax(y_prob_vec, axis=1)

    fu.log(workdir, f'[{i}] Done predict()',
                   f'mem, ({fu.get_my_memory()})')
    correct = len([i for i, _ in enumerate(actuals)
              if actuals[i] == predictions[i]])
    acc = correct/len(actuals)
    fu.log(workdir, f'[{i}], acc={acc}')

    bundle_loc = f'{workdir}/bundle_{i}.joblib'
    logloss = fu.big_logloss(actuals, y_prob=y_prob_vec, 
                             labels= list(range(len(labels))))
    fu.log(workdir, f'[{i}] Done  done fu.big_logloss() logloss={logloss}',
                  f'mem, ({fu.get_my_memory()})')

    joblib.dump({
     'model': xgb_model,
     'notebook': '2020-06-21.ipynb',
     'num_round': num_round,
     'metrics': {
                 'accuracy': acc,   
                 'validation_logloss': logloss,
                 'confusion_matrix': confusion_matrix(
                     actuals, predictions)
                 },
     'timestamp': fu.utc_ts(),
     'input_params': params,
    }, bundle_loc)
    fu.log(workdir, f'[{i}] wrote bundle {bundle_loc}')




workdir:  /opt/program/artifacts/2020-06-21T213257Z
[21:33:36] 1x78 matrix with 16000 entries loaded from /opt/program/artifacts/2020-06-21T213257Z/dtrain.txt#dtrain.cache
[21:33:36] WARNING: src/learner.cc:658: Tree method is automatically set to 'approx' since external-memory data matrix is used.
[21:33:36] SparsePageSource: Finished writing to dtrain.cache
[21:33:36] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[21:33:36] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[21:33:36] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[21:33:36] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[21:33:36] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[21:33:36] INFO: src/tree/updater_prune.cc:74: tree p

[21:33:37] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[21:33:37] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[21:33:37] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[21:33:37] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[21:33:37] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[21:33:37] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[21:33:37] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[21:33:37] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[21:33:37] INFO: src/tree/updater_prune.cc:74: tree pruning end,

[21:33:37] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[21:33:37] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[21:33:37] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[21:33:37] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[21:33:37] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[21:33:37] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[21:33:37] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[21:33:37] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[21:33:37] INFO: src/tree/updater_prune.cc:74: tree pruning end,

[21:33:37] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[21:33:37] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[21:33:37] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[21:33:37] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[21:33:37] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[21:33:37] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[21:33:37] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[21:33:37] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[21:33:37] INFO: src/tree/updater_prune.cc:74: tree pruning end,

[21:33:37] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[21:33:37] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[21:33:37] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[21:33:37] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[21:33:37] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[21:33:37] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[21:33:37] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[21:33:37] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[21:33:37] INFO: src/tree/updater_prune.cc:74: tree pruning end,

...

[21:36:19] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[21:36:19] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[21:36:19] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[21:36:19] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[21:36:19] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[21:36:19] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[21:36:19] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[21:36:19] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[21:36:19] INFO: src/tree/updater_prune.cc:74: tree pruning end,

[21:36:19] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[21:36:19] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[21:36:19] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[21:36:19] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[21:36:19] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[21:36:19] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[21:36:19] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[21:36:19] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[21:36:19] INFO: src/tree/updater_prune.cc:74: tree pruning end,

[21:36:20] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[21:36:20] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[21:36:20] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[21:36:20] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[21:36:20] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[21:36:20] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[21:36:20] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[21:36:20] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[21:36:20] INFO: src/tree/updater_prune.cc:74: tree pruning end,

[21:36:20] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[21:36:20] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[21:36:20] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[21:36:20] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[21:36:20] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[21:36:20] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[21:36:20] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[21:36:20] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[21:36:20] INFO: src/tree/updater_prune.cc:74: tree pruning end,

[21:36:20] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[21:36:20] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[21:36:20] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[21:36:20] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[21:36:20] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[21:36:20] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[21:36:20] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[21:36:20] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[21:36:20] INFO: src/tree/updater_prune.cc:74: tree pruning end,

#### log dump of num_round=2 attempt
* `Wall time: 2min 37s` ..
```
(pandars3) $ tail -f artifacts/2020-06-21T212459Z/work.log 
2020-06-21 21:24:59Z, Starting, mem, ({'pmem': '36.4', 'rss': '0.712 GiB'})
2020-06-21 21:25:00Z, [0] Done preprocessing, mem, ({'pmem': '36.4', 'rss': '0.712 GiB'})
2020-06-21 21:25:38Z, [0] Saved train data: /opt/program/artifacts/2020-06-21T212459Z/dtrain.txt
2020-06-21 21:25:38Z, [0] Done xgb.train, mem, ({'pmem': '37.3', 'rss': '0.73 GiB'})
2020-06-21 21:25:39Z, [0] Done transforming test data
2020-06-21 21:26:15Z, [0] Done saving to /opt/program/artifacts/2020-06-21T212459Z/dtest.txt
2020-06-21 21:26:16Z, [0] Done predict(), mem, ({'pmem': '59.7', 'rss': '1.166 GiB'})
2020-06-21 21:26:17Z, [0], acc=0.02239938535669231
2020-06-21 21:26:18Z, [0] Done  done fu.big_logloss() logloss=0.0, mem, ({'pmem': '60.1', 'rss': '1.175 GiB'})
2020-06-21 21:26:19Z, [0] wrote bundle /opt/program/artifacts/2020-06-21T212459Z/bundle_0.joblib
2020-06-21 21:26:20Z, [1] Done preprocessing, mem, ({'pmem': '60.9', 'rss': '1.189 GiB'})
2020-06-21 21:26:57Z, [1] Saved train data: /opt/program/artifacts/2020-06-21T212459Z/dtrain.txt
2020-06-21 21:26:57Z, [1] Done xgb.train, mem, ({'pmem': '62.0', 'rss': '1.211 GiB'})
2020-06-21 21:26:58Z, [1] Done transforming test data
2020-06-21 21:27:33Z, [1] Done saving to /opt/program/artifacts/2020-06-21T212459Z/dtest.txt
2020-06-21 21:27:34Z, [1] Done predict(), mem, ({'pmem': '62.4', 'rss': '1.219 GiB'})
2020-06-21 21:27:35Z, [1], acc=0.022382786193290143
2020-06-21 21:27:36Z, [1] Done  done fu.big_logloss() logloss=0.0, mem, ({'pmem': '62.4', 'rss': '1.22 GiB'})
2020-06-21 21:27:36Z, [1] wrote bundle /opt/program/artifacts/2020-06-21T212459Z/bundle_1.joblib
```


In [10]:
# Reran the logloss from above b/c i had my bug there from earlier..
fu.big_logloss(actuals, y_prob=y_prob_vec, 
                             labels= list(range(len(labels))))

3.9889841079711914

#### log dump w num_round=100
* Hmm although `100` vs `2` rounds took `20 seconds` vs `<1 sec` , the acc and log loss are identical, so something w/ parameters definitely different.. between here and "2020-06-20"

```
(pandars3) $ tail -f artifacts/2020-06-21T213257Z/work.log 
2020-06-21 21:32:57Z, Starting, mem, ({'pmem': '32.8', 'rss': '0.64 GiB'})
2020-06-21 21:32:58Z, [0] Done preprocessing, mem, ({'pmem': '34.4', 'rss': '0.672 GiB'})
2020-06-21 21:33:36Z, [0] Saved train data: /opt/program/artifacts/2020-06-21T213257Z/dtrain.txt
2020-06-21 21:33:57Z, [0] Done xgb.train, mem, ({'pmem': '36.2', 'rss': '0.707 GiB'})
2020-06-21 21:33:58Z, [0] Done transforming test data
2020-06-21 21:34:35Z, [0] Done saving to /opt/program/artifacts/2020-06-21T213257Z/dtest.txt
2020-06-21 21:35:12Z, [0] Done predict(), mem, ({'pmem': '58.6', 'rss': '1.145 GiB'})
2020-06-21 21:35:12Z, [0], acc=0.02239938535669231
2020-06-21 21:35:15Z, [0] Done  done fu.big_logloss() logloss=3.9889841079711914, mem, ({'pmem': '58.9', 'rss': '1.15 GiB'})
2020-06-21 21:35:15Z, [0] wrote bundle /opt/program/artifacts/2020-06-21T213257Z/bundle_0.joblib
2020-06-21 21:35:16Z, [1] Done preprocessing, mem, ({'pmem': '59.8', 'rss': '1.169 GiB'})
2020-06-21 21:35:55Z, [1] Saved train data: /opt/program/artifacts/2020-06-21T213257Z/dtrain.txt
2020-06-21 21:36:21Z, [1] Done xgb.train, mem, ({'pmem': '61.1', 'rss': '1.194 GiB'})
2020-06-21 21:36:22Z, [1] Done transforming test data
2020-06-21 21:36:59Z, [1] Done saving to /opt/program/artifacts/2020-06-21T213257Z/dtest.txt
2020-06-21 21:37:35Z, [1] Done predict(), mem, ({'pmem': '62.1', 'rss': '1.212 GiB'})
2020-06-21 21:37:35Z, [1], acc=0.022382786193290143
2020-06-21 21:37:36Z, [1] Done  done fu.big_logloss() logloss=3.9889841079711914, mem, ({'pmem': '62.1', 'rss': '1.213 GiB'})
2020-06-21 21:37:37Z, [1] wrote bundle /opt/program/artifacts/2020-06-21T213257Z/bundle_1.joblib

```
